In [ ]:
from jupyter_innotater import *
import numpy as np, os

### Image Filenames and Bounding Boxes

In [ ]:
foodfns = sorted(os.listdir('./foods/'))
targets = np.zeros((len(foodfns), 4), dtype='int') # (x,y,w,h) for each data row

Innotater( ImageInnotation(foodfns, path='./foods'), BoundingBoxInnotation(targets) )

Press 'n' or 'p' to move to next or previous image in the Innotater above.

In [ ]:
targets

In [ ]:
# Write our newly-input bounding box data to disk - will be lost otherwise
# If pandas not installed, please just ignore this cell
import pandas as pd
df = pd.DataFrame(targets, columns=['x','y','w','h'])
df.insert(0,'filename', foodfns)
df.to_csv('./bounding_boxes.csv')
df

### Numpy Image Data and Multi-classification

In [ ]:
try:
    import cv2
    foods = [cv2.imread('./foods/'+f) for f in foodfns]

except ModuleNotFoundError:
    print("OpenCV2 is not installed, so just using filenames like before - Innotater will understand")
    foods = ['./foods/'+f for f in foodfns]

classes = ['vegetable', 'biscuit', 'fruit']
targets = [0] * len(foodfns)

In [ ]:
w2 = Innotater(
        ImageInnotation(foods, name='Food'), 
        MultiClassInnotation(targets, name='FoodType', classes=classes, desc='Food Type')
)
display(w2)

In [ ]:
targets

In [ ]:
# Convert targets from a 1-dim array to one-hot representation - Innotater works with that just as well
onehot_targets = np.squeeze(np.eye(len(classes))[np.array(targets).reshape(-1)]); onehot_targets

In [ ]:
Innotater(
    ImageInnotation(foods, name='Food'), 
    MultiClassInnotation(onehot_targets, name='FoodType', classes=classes, desc='Food Type')
)

### Filenames and binary classification

Set image to display at a smaller width to make it more manageable - but bounding box co-ordinates would be relative to the unzoomed image.

In [ ]:
isfruit_targets = (np.array(targets) == 2).astype('int')
Innotater( ImageInnotation(foodfns, path='./foods', width=300),
                BinaryClassInnotation(isfruit_targets, name='Is Fruit')
              )

In [ ]:
isfruit_targets

### Image Filenames and Binary Classification plus Bounding Boxes

Use indexes attribute to limit display just to the fruits where we want to add bounding boxes. Drop the indexes property if you also want to be able to check non-fruits.

In [ ]:
bboxes = np.zeros((len(foodfns),4), dtype='int')
isfruits = np.expand_dims(isfruit_targets, axis=-1)

suspected_fruits = isfruits == 1 # Or you can specify an array/list of int indices

Innotater(
        ImageInnotation(foodfns, name='Food', path='./foods'), 
        [ BinaryClassInnotation(isfruits, name='Is Fruit'),
          BoundingBoxInnotation(bboxes, name='bbs', source='Food', desc='Food Type') ],
    indexes = suspected_fruits
)

In [ ]:
result = np.concatenate([isfruits,bboxes], axis=-1); result

### Image versus Image and Binary Classification

In [ ]:
targets = np.array([[1,0]] * 5) # One-hot format, defaulting to 0 class
lfoods = foods[:5]
rfoods = lfoods.copy()
rfoods.reverse()

Innotater([ImageInnotation(lfoods, name='Food 1'), ImageInnotation(rfoods, name='Food 2')], 
        [BinaryClassInnotation(targets, name='Are Equal')])

In [ ]:
targets

### Text Data - sentiment classification
Movie reviews. In this example, numbers prefix the class names so you can keep input focus in the listbox and press 0, 1, or 2 to select the sentiment label, then press 'n' to advance to the next review (or 'p' to go back).

In [ ]:
reviews = ['I really liked this movie', 'It was OK', 'Do not watch!', 'Was worth trying it']
sentiments = [1] * len(reviews)
sentiment_classes = ['0 - Positive', '1 - Neutral', '2 - Negative']

Innotater(TextInnotation(reviews), MultiClassInnotation(sentiments, classes=sentiment_classes))

In [ ]:
list(zip(reviews, [sentiment_classes[s] for s in sentiments]))